In [1]:
import pandas as pd
import numpy as np

In [2]:
names = "Muestra #0,Muestra #1,Muestra #2,Muestra #3,Muestra #4,Muestra #5,Muestra #6,Muestra #7,Muestra #8,Muestra #9,Muestra #10,Muestra #11,Muestra #12,Muestra #13,Muestra #14,Muestra #15,Muestra #16,Muestra #17,Muestra #18,Muestra #19,Muestra #20,Muestra #21,Muestra #22,Muestra #23,Muestra #24,Muestra #25,Muestra #26,Muestra #27,Muestra #28,Muestra #29,Muestra #30,Muestra #31"
values = "Ambas son equivalentes,A es mucho mejor que B,A es mucho mejor que B,A es mucho mejor que B,Ambas son equivalentes,A es mucho mejor que B,Ambas son equivalentes,Ambas son equivalentes,A es mucho mejor que B,A es mucho mejor que B,Ambas son equivalentes,A es mucho mejor que B,A es mucho mejor que B,Ambas son equivalentes,Ambas son equivalentes,A es mucho mejor que B,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,Ambas son equivalentes,A es mucho mejor que B,Ambas son equivalentes,A es mucho mejor que B,Ambas son equivalentes,A es mucho mejor que B"

names = names.split(',')
values = values.split(',')

In [3]:
index = {
    "A es mucho mejor que B": -2,
    "A es ligeramente mejor que B": -1,
    "Ambas son equivalentes": 0,
    "B es ligeramente mejor que A": 1,
    "B es mucho mejor que A": 2,
}

In [4]:
import json
with open('D:/Thesis/exp_out/result.json', 'r') as file:
    known_values = json.load(file)

In [5]:
results = {}

for name, value in zip(names, values):
    print(name, ' - ', value)
    for key in known_values:
        if key.startswith(f"{name} - "):
            a = known_values[key]['a']
            b = known_values[key]['b']

            if index[value] < 0:
                results[a] = results.get(a, 0) + 1

Muestra #0  -  Ambas son equivalentes
Muestra #1  -  A es mucho mejor que B
Muestra #2  -  A es mucho mejor que B
Muestra #3  -  A es mucho mejor que B
Muestra #4  -  Ambas son equivalentes
Muestra #5  -  A es mucho mejor que B
Muestra #6  -  Ambas son equivalentes
Muestra #7  -  Ambas son equivalentes
Muestra #8  -  A es mucho mejor que B
Muestra #9  -  A es mucho mejor que B
Muestra #10  -  Ambas son equivalentes
Muestra #11  -  A es mucho mejor que B
Muestra #12  -  A es mucho mejor que B
Muestra #13  -  Ambas son equivalentes
Muestra #14  -  Ambas son equivalentes
Muestra #15  -  A es mucho mejor que B
Muestra #16  -  Ambas son equivalentes
Muestra #17  -  Ambas son equivalentes
Muestra #18  -  Ambas son equivalentes
Muestra #19  -  Ambas son equivalentes
Muestra #20  -  Ambas son equivalentes
Muestra #21  -  Ambas son equivalentes
Muestra #22  -  Ambas son equivalentes
Muestra #23  -  Ambas son equivalentes
Muestra #24  -  Ambas son equivalentes
Muestra #25  -  Ambas son equivalen

In [6]:
results

{'Threshold Coefficients at 0.1': 9,
 'Gaussian Scale Mask with center_scale=N/2, sigma=1.0': 3}

In [7]:
tie = 18
thr_wins = 9
gauss_wins = 3
total = len(values)

In [8]:
tie/total

0.5625

In [9]:
thr_wins/total

0.28125

In [10]:
gauss_wins/total

0.09375

In [12]:
from scipy.stats import chi2

def mcnemar_test(table, correction=True, alpha=0.05):
    """
    Realiza la prueba de McNemar para datos pareados categóricos.

    Parámetros:
    -----------
    table : array-like de 2x2
        Tabla de contingencia con las frecuencias observadas:
        [[A, B],
         [C, D]]
        Donde:
        - A: Casos donde ambos métodos son positivos (rara vez usado).
        - B: Casos donde el Método 1 es positivo y el Método 2 negativo.
        - C: Casos donde el Método 1 es negativo y el Método 2 positivo.
        - D: Casos donde ambos métodos son negativos.

    correction : bool, optional (default=True)
        Si aplicar la corrección de continuidad de Edwards.

    alpha : float, optional (default=0.05)
        Nivel de significancia para la interpretación.

    Retorna:
    --------
    dict
        Diccionario con los resultados:
        - 'statistic': Valor del estadístico.
        - 'pvalue': Valor-p bilateral.
        - 'significant': True si pvalue < alpha.
        - 'interpretation': Explicación en lenguaje natural.
    """
    # Validar la tabla de entrada
    table = np.asarray(table)
    if table.shape != (2, 2):
        raise ValueError("La tabla debe ser de 2x2.")
    
    B = table[0, 1]  # Método 1 gana
    C = table[1, 0]  # Método 2 gana
    
    # Calcular el estadístico con/sin corrección
    if correction:
        statistic = (abs(B - C) - 1)**2 / (B + C)
    else:
        statistic = (B - C)**2 / (B + C)
    
    # Calcular el valor-p (distribución chi-cuadrado con 1 gl)
    pvalue = 1 - chi2.cdf(statistic, df=1)
    
    # Interpretación
    total_discordant = B + C
    if total_discordant == 0:
        interpretation = "No hay casos discordantes (todos son empates)."
    else:
        winner = "Método 1" if B > C else "Método 2"
        percent_diff = 100 * abs(B - C) / total_discordant
        interpretation = (
            f"Preferencia por {winner} ({percent_diff:.1f}% de diferencia en casos discordantes). "
            f"La diferencia {'es' if pvalue < alpha else 'no es'} estadísticamente significativa."
        )
    
    return {
        'statistic': statistic,
        'pvalue': pvalue,
        'significant': pvalue < alpha,
        'interpretation': interpretation
    }

In [14]:
# Tabla de contingencia:
#                  | Gaussian No Gana | Gaussian Gana |
# -----------------------------------------------
# Thresholding Gana |        9        |       0       |
# Thresholding No Gana |       18       |       3       |

# Nota: Reorganizamos para que las discordancias estén en B y C.
table = np.array([
    [9,  0],  # Thresholding gana (pero Gaussian no)
    [18, 3]   # Thresholding no gana (pero Gaussian sí en 3 casos)
])

# Llamar a la función
result = mcnemar_test(table[:, [1, 0]], correction=True)  # Reordenamos columnas para discordancias

print(f"Estadístico de McNemar: {result['statistic']:.4f}")
print(f"Valor-p: {result['pvalue']:.4f}")
print(f"Interpretación: {result['interpretation']}")

Estadístico de McNemar: 2.0833
Valor-p: 0.1489
Interpretación: Preferencia por Método 1 (50.0% de diferencia en casos discordantes). La diferencia no es estadísticamente significativa.
